In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\vivek\\Documents\\GSC\\chatbot\\law_chatbot_gen-ai\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vivek\\Documents\\GSC\\chatbot\\law_chatbot_gen-ai'

In [5]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 14248


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:

#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\vivek\AppData\Local\Temp\ipykernel_19432\67698624.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\vivek\anaconda3\envs\chatbot_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY2')

In [17]:
import pinecone
import os

# Initialize Pinecone the correct way
pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY2"))

# Define the index name
index_name = "lawbot2"

# List existing indexes (optional)
print(pc.list_indexes())


{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'lawbot-t6cl0l6.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'lawbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [18]:
import os
os.environ["PINECONE_API_KEY2"] = PINECONE_API_KEY


In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

ValueError: Index 'lawbot2' not found in your Pinecone project. Did you mean one of the following indexes: lawbot